##### Copyright 2024 Google LLC.

In [44]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Caching Quickstart with REST

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/rest/Caching_REST.ipynb"><img src="../../images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>


If you want to quickly try out the [Context Caching](https://ai.google.dev/gemini-api/docs/caching?lang=rest) with the Gemini API, you can use `curl` commands to call the methods in the REST API. Context caching is a way to save on requests costs when a substantial initial context is referenced repeatedly by shorter requests.


If you have never used the Gemini REST API, it is strongly recommended to start with the [Prompting quickstart](https://github.com/google-gemini/cookbook/blob/main/quickstarts/rest/Prompting_REST.ipynb) first.

This notebook contains `curl` commands you can run in Google Colab, or copy to your terminal.

To run this notebook, your API key must be stored it in a Colab Secret named GOOGLE_API_KEY. If you are running in a different environment, you can store your key in an environment variable. See [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) to learn more.

In [45]:
import os
from google.colab import userdata

In [46]:
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

## Caching content

Let's start by getting the transcript from the Apollo 11 mission.


In [47]:
!wget https://storage.googleapis.com/generativeai-downloads/data/a11.txt

--2024-07-11 17:55:31--  https://storage.googleapis.com/generativeai-downloads/data/a11.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.207, 74.125.20.207, 108.177.98.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847790 (828K) [text/plain]
Saving to: ‘a11.txt.2’

a11.txt.2           100%[===================>] 827.92K  --.-KB/s    in 0.008s  

2024-07-11 17:55:31 (103 MB/s) - ‘a11.txt.2’ saved [847790/847790]



Now you need to reencode it to base-64, so let's prepare the whole [cachedContent](https://ai.google.dev/api/rest/v1beta/cachedContents#resource:-cachedcontent) while you're at it.

Note that you have to use a stable model with fixed versions (`gemini-1.5-flash-001` in this case). That's why you had to add a version postfix ("`-001`" in this case). You'll get a `Model [...] not found or does not support caching` error if you forget to do so.


In [48]:
%%bash

echo '{
  "model": "models/gemini-1.5-flash-001",
  "contents":[
    {
      "parts":[
        {
          "inline_data": {
            "mime_type":"text/plain",
            "data": "'$(base64 -w0 a11.txt)'"
          }
        }
      ],
    "role": "user"
    }
  ],
  "systemInstruction": {
    "parts": [
      {
        "text": "You are an expert at analyzing transcripts."
      }
    ]
  },
  "ttl": "600s"
}' > request.json

We can now create the cached content.

In [59]:
%%bash

curl -X POST "https://generativelanguage.googleapis.com/v1beta/cachedContents?key=$GOOGLE_API_KEY" \
 -H 'Content-Type: application/json' \
 -d @request.json > cache.json

 cat cache.json

{
  "name": "cachedContents/lf0nt062ulc1",
  "model": "models/gemini-1.5-flash-001",
  "createTime": "2024-07-11T18:02:48.257891Z",
  "updateTime": "2024-07-11T18:02:48.257891Z",
  "expireTime": "2024-07-11T18:07:47.635193373Z",
  "displayName": "",
  "usageMetadata": {
    "totalTokenCount": 323383
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1104k    0   307  100 1104k    138   499k  0:00:02  0:00:02 --:--:--  499k


You will need it for the next commands so save the name of the cache.

You're using a text file to save the name here beacuse of colab constrainsts but you could also simply use a variable.

In [57]:
%%bash

CACHE_NAME=$(cat cache.json | grep '"name":' | cut -d '"' -f 4 | head -n 1)

echo $CACHE_NAME > cache_name.txt

cat cache_name.txt

cachedContents/qidqwuaxdqz4


## Listing caches
Since caches have a reccuring cost it's a good idea to keep an eye on them. It can also be useful if you need to find their name.

In [60]:
!curl "https://generativelanguage.googleapis.com/v1beta/cachedContents?key=$GOOGLE_API_KEY"

{
  "cachedContents": [
    {
      "name": "cachedContents/lf0nt062ulc1",
      "model": "models/gemini-1.5-flash-001",
      "createTime": "2024-07-11T18:02:48.257891Z",
      "updateTime": "2024-07-11T18:02:48.257891Z",
      "expireTime": "2024-07-11T18:07:47.635193373Z",
      "displayName": "",
      "usageMetadata": {
        "totalTokenCount": 323383
      }
    },
    {
      "name": "cachedContents/qidqwuaxdqz4",
      "model": "models/gemini-1.5-flash-001",
      "createTime": "2024-07-11T18:02:30.516233Z",
      "updateTime": "2024-07-11T18:02:30.516233Z",
      "expireTime": "2024-07-11T18:07:29.803448998Z",
      "displayName": "",
      "usageMetadata": {
        "totalTokenCount": 323383
      }
    }
  ]
}


## Using cached content when prompting

Prompting using cached content is the same as what is illustrated in the [Prompting quickstart](https://github.com/google-gemini/cookbook/blob/main/quickstarts/rest/Prompting_REST.ipynb) except you're adding a `"cachedContent"` value which is the name of the cache you saved earlier.

In [51]:
%%bash

curl -X POST "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-001:generateContent?key=$GOOGLE_API_KEY" \
-H 'Content-Type: application/json' \
-d '{
      "contents": [
        {
          "parts":[{
            "text": "Please summarize this transcript"
          }],
          "role": "user"
        },
      ],
      "cachedContent": "'$(cat cache_name.txt)'"
    }'

{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "This is a transcript of the air-to-ground voice transmission between the Apollo 11 crew and Mission Control during their journey to the moon and back. The transcript covers the period from the launch of the Saturn V rocket until the splashdown of the command module in the Pacific Ocean. \n\nThe transcript documents a fascinating conversation between the astronauts and Mission Control, highlighting the various activities of the mission. It covers:\n\n* **Launch and Trans-Lunar Injection:** The launch sequence, staging of the rocket, and the critical Trans-Lunar Injection (TLI) maneuver that sent the Apollo 11 spacecraft towards the moon.\n* **Orbit and Docking:** The crew's actions in earth orbit, their successful docking with the Lunar Module (LM), and the transfer of the LM to the docking port of the command module. \n* **Lunar Surface Preparations:** Communications checks, pre-flight prep

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3042    0  2817  100   225    117      9  0:00:25  0:00:24  0:00:01   580


As you can see, among the 323699 tokens, 323383 were cached (and thus less expensive) and only 311 were from the prompt.

As of July 11th 2024, this would mean about half the price of a normal prompt.


## Optional: Updating a cache
If you need to update a cache, to chance its content, or just extend its longevity, just use `PATCH`:

In [62]:
%%bash

curl -X PATCH "https://generativelanguage.googleapis.com/v1beta/$(cat cache_name.txt)?key=$GOOGLE_API_KEY" \
 -H 'Content-Type: application/json' \
 -d '{"ttl": "300s"}'

{
  "name": "cachedContents/qidqwuaxdqz4",
  "model": "models/gemini-1.5-flash-001",
  "createTime": "2024-07-11T18:02:30.516233Z",
  "updateTime": "2024-07-11T18:05:38.781423Z",
  "expireTime": "2024-07-11T18:10:38.759996261Z",
  "displayName": "",
  "usageMetadata": {
    "totalTokenCount": 323383
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   322    0   307  100    15    822     40 --:--:-- --:--:-- --:--:--   863


## Deleting cached content

The cache has a small recurring storage cost so by default it is only saved for an hour. In this case you even set it up for a shorter amont of time (using `"ttl"`) of 10mn.

Still, if you don't need you cache anymore, it is good practice to delete it proactively.

In [63]:
!curl -X DELETE "https://generativelanguage.googleapis.com/v1beta/$(cat cache_name.txt)?key=$GOOGLE_API_KEY"

{}


## Next steps

If you want to know more about the caching REST APIs, you can check the full [API specifications](https://ai.google.dev/api/rest/v1beta/cachedContents) and the [caching documentation](https://ai.google.dev/gemini-api/docs/caching).

The Gemini API also has configurable safety settings. Learn more [here](https://github.com/google-gemini/cookbook/blob/main/quickstarts/rest/Safety_REST.ipynb).